In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)

## read reverse mortgage data

In [ ]:
alameda_rm_all = pd.read_csv('./reverse_mortgage_data/alameda_rm_all.csv', dtype={
    'county_code': str,
    'census_tract': str
})
alameda_rm_all.info()

## read census data and filter to county

In [ ]:
census_all = pd.read_csv('./census_data/ACS2019_filtered0623.csv', dtype={
    'census_tract': str,
    'county_code': str,
    'census_tract_np': str,
    'census_tract_period': str
})
census_all.info()

In [ ]:
census_alameda = census_all[census_all['county_code'] == '06001'].copy()

In [ ]:
census_alameda.info()

## merge lender list

In [ ]:
# read lender list

lender_list_2018 = pd.read_csv('./raw_data/lei_agency_name_2018.csv')

In [ ]:
lender_list_2018.info()

In [ ]:
# merge by lei

alameda_rm_all

In [ ]:
lender_list_2018

In [ ]:
alameda_rm_full = pd.merge(
    alameda_rm_all,
    lender_list_2018,
    on='lei',
    how='left',
    validate='m:1'
)

In [ ]:
alameda_rm_full.info(verbose=True, show_counts=True)

In [ ]:
alameda_rm_name = alameda_rm_full.drop(columns= ['activity_year_y', 'calendar_quarter', 'lar_count']).copy()

In [ ]:
alameda_rm_name.head()

## group by census tracts

In [ ]:
# filter by action_taken = 1

alameda_rm_originated = alameda_rm_name[alameda_rm_name['action_taken'] == 1]

In [ ]:
alameda_rm_originated.info(verbose=True, show_counts=True)

In [ ]:
# group by census_tract

alameda_census_counts = alameda_rm_originated.groupby(['census_tract']).count()

In [ ]:
alameda_census_counts = alameda_census_counts[['lei']].copy()

In [ ]:
alameda_census_counts = alameda_census_counts.reset_index()

In [ ]:
alameda_census_counts = alameda_census_counts.sort_values(by=['lei'], ascending=False).copy()
alameda_census_counts.rename(columns={'lei': 'rm_no'}, inplace=True)

In [ ]:
alameda_census_counts = alameda_census_counts.reset_index(drop=True)

In [ ]:
alameda_census_counts

## merge with census data

In [ ]:
census_alameda

In [ ]:
alameda_census_rm = pd.merge(
    alameda_census_counts,
    census_alameda,
    on='census_tract',
    how='left',
    validate='1:1'
)

In [ ]:
alameda_census_rm.sample(10)

In [ ]:
alameda_census_rm.info(verbose=True)

In [ ]:
# subset to the columns that I need

alameda_census_rm = alameda_census_rm[['census_tract', 'rm_no', 'county_code', 'county', 'state_code', 'census_tract_np', 'census_tract_period', 'total_population', '62yr_pp_pct', 'median_age', 'hispanic_pp_pct', 'non_hispanic_black_pp_pct']].copy()

In [ ]:
alameda_census_rm

In [ ]:
# calculate rm per 1000 people

alameda_census_rm['_rm_per_1000'] = (
    alameda_census_rm['rm_no'] / alameda_census_rm['total_population'] * 1000
)

In [ ]:
alameda_census_rm.info()

In [ ]:
# get the column names

cols = list(alameda_census_rm.columns.values)
cols

In [ ]:
# reorder the columns

alameda_census_rm = alameda_census_rm[[
    'census_tract', 
    '_rm_per_1000',
    'rm_no',
    'county_code',
    'county',
    'state_code',
    'census_tract_np',
    'census_tract_period',
    'total_population',
    '62yr_pp_pct',
    'median_age',
    'hispanic_pp_pct',
    'non_hispanic_black_pp_pct',
 ]]

In [ ]:
# sort by reverse mortgages originated per 1000 people

alameda_census_rm.sort_values(by='_rm_per_1000', ascending=False)

## export to csv

In [ ]:
alameda_census_rm.to_csv('./reverse_mortgage_data/alameda_census_rm.csv', index=False)